In [ ]:
import numpy as np
from scipy.special import expit
import matplotlib.pyplot as plt

np.random.seed(seed=0)

# Data

Linearly separable dataset consisting of datapoints generated from two Gaussian distributions.

In [ ]:
mean_1 = [-5, 2]
cov_1 = [[1,0.5],[0.5,3]]
gaussian_1 = np.random.multivariate_normal(mean_1, cov_1, 50)

mean_2 = [5, -2]
cov_2 = [[1,0.5],[0.5,3]]
gaussian_2 = np.random.multivariate_normal(mean_2, cov_2, 50)

plt.scatter(gaussian_1[:,0], gaussian_1[:,1])
plt.scatter(gaussian_2[:,0], gaussian_2[:,1])
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()

### Sort data into labeled dataset

dataset[0,:] -> input vector of index 0

dataset[:,1] -> x_1

dataset[:,2] -> x_2

In [ ]:
data_1 = np.zeros((gaussian_1.shape[0], 3))
data_1[:,0:2] = gaussian_1

data_2 = np.ones((gaussian_2.shape[0], 3))
data_2[:,0:2] = gaussian_2

dataset = np.vstack((data_1, data_2))
dataset = np.random.permutation(dataset)

N = dataset.shape[0]

### Fixed bias vectors

In [ ]:
phis = np.zeros((6, N))
phis[0,:] = dataset[:,0]                 # x_1
phis[1,:] = dataset[:,0]**2              # x_1^2
phis[2,:] = dataset[:,1]                 # x_2
phis[3,:] = dataset[:,1]**2              # x_2^2
phis[4,:] = dataset[:,0] * dataset[:,1]  # x_1 * x_2
phis[5,:] = 1                            # bias

D = phis.shape[0]

ts = np.zeros((1,N))
ts[0,:] = dataset[:,2]

In [ ]:
print(phis.shape)
print(D)
print(ts.shape)

# Initialize variables

# What is this local variational inference functional really?

In [ ]:
def lambda_func(xi):
    return (expit(xi) - 0.5) / (2*xi + 1e-24)

# Expectations

In [ ]:
def gamma_expectation(a_N, b_N):
    return a_N / b_N

#def gaussian_quadratic_expectation(mu_N, sigma_N):
#    #return sigma_N + np.matmul(mu_N, mu_N.T)
#    return sigma_N + np.matmul(mu_N.T, mu_N)

def gaussian_quadratic_expectation_wrt_w(mu_N):
    return np.matmul(mu_N.T, mu_N)

# Maximization (re-estimation) equations

In [ ]:
#######################################################
#  Distribution 'q(w) = Gaussian(w | mu_N, sigma_N)'
#######################################################

def maximization_mu(phis, ts, sigma_N):
    
    # Vector container for summing individual samples
    sample_vec_sum = np.zeros((phis.shape[0], 1))
    
    # Compute and sum each sample individually (TODO: vectorization)
    for i in range(phis.shape[1]):
        
        # Vectors for sample 'i'
        phi_i = phis[:,i:i+1]  # [D,1]
        t_i = ts[:,i].item()   # scalar
        
        sample_vec_sum += (t_i - 0.5) * phi_i
    
    mu_N = np.matmul(sigma_N, sample_vec_sum)
    
    return mu_N

def maximization_sigma_inv(expectation_alpha, xis, phis):
    
    # Vector container for summing individual samples
    sample_vec_sum = np.zeros((D,D))
    
    # Compute and sum each sample individually (TODO: vectorization)
    for i in range(phis.shape[1]):
        
        # Vectors for sample 'i'
        phi_i = phis[:,i:i+1]   # [D,1]
        xi_i = xis[:,i].item()  # scalar
        
        sample_vec_sum += lambda_func(xi_i) * np.matmul(phi_i, phi_i.T)
        
    sigma_inv = expectation_alpha * np.eye(D) + 2.0 * sample_vec_sum

    return sigma_inv


####################################################
#  Distribution 'q(alpha) = Gam(alpha | a_N, b_N)  #
####################################################

def maximization_an(a_0, D):
    return a_0 + D

def maximization_bn(b_0, expectation_quadratic_w):
    return b_0 + 0.5 * expectation_quadratic_w


###############
#  Xi update
###############

def xi_update(phi, mu_N, sigma_N):
    # xi = phi.T * (sigma_N + mu_N * mu_N.T) * phi
    A = sigma_N + np.matmul(mu_N, mu_N.T)
    xi = np.matmul(np.matmul(phi.T, A), phi)
    xi = np.sqrt(xi)
    return xi

def xis_update(phis, mu_N, sigma_N):
    
    N = phis.shape[1]
    
    # Vector container for storing updated 'xi' values
    xis_new = np.zeros((1,N))
    
    # Compute each sample individually
    for i in range(N):
        
        # Vector for sample 'i'
        phi_i = phis[:,i:i+1]   # [D,1]
        
        xis_new[0,i] = xi_update(phi_i, mu_N, sigma_N)
    
    return xis_new


# Expectation Maximization iteration

In [ ]:
# Initialize variables

# Hyperpriors for Gamma distribution
a_0 = 1.0
b_0 = 1.0

a_N = a_0
b_N = b_0

#mu_N = 0.1*np.ones((D,1))
mu_N = 0.01*np.random.random((D,1))
#sigma_N = 0.1*np.ones((D,D))
sigma_N = 0.01*np.random.random((D,D))

# Sample-specific latent variables 'xi'
xis = np.ones((1, N))

old_vals = np.zeros(7)

deltas = np.zeros((7,100))

for iter_idx in range(100):
    
    # Expectation step
    expectation_alpha = gamma_expectation(a_N, b_N)                             # scalar
    expectation_quadratic_w_wrt_w = gaussian_quadratic_expectation_wrt_w(mu_N)  # scalar ?
    
    # Maximization step
    a_N = maximization_an(a_0, D)                                               # scalar
    b_N = maximization_bn(b_0, expectation_quadratic_w_wrt_w)                   # scalar
    sigma_N_inv = maximization_sigma_inv(expectation_alpha, xis, phis)          # [D,D]
    sigma_N = np.linalg.inv(sigma_N_inv)                                        # [D,D]
    mu_N = maximization_mu(phis, ts, sigma_N)                                   # [D,1]
    xis = xis_update(phis, mu_N, sigma_N)                                       # [1,N]
    

In [ ]:
x = [0, 0]



def predictive_posterior_prob(x, mu_n, sigma_N):
    
    phi_test = np.zeros((6,1))
    phi_test[0,:] = x[0]
    phi_test[1,:] = x[0]**2
    phi_test[2,:] = x[1]
    phi_test[3,:] = x[1]**2
    phi_test[4,:] = x[0] * x[1]
    phi_test[5,:] = 1

    mu_a = np.matmul(mu_N.T, phi_test).item()

    #rint(mu_a)

    sigma_a_2 = np.matmul(np.matmul(phi_test.T, sigma_N), phi_test).item()

    #rint(sigma_a_2)

    kappa = np.power(1.0 + np.pi/8.0 * sigma_a_2,-0.5)

    #rint(kappa)

    p = expit(kappa*mu_a)
    
    #rint(p)
    
    return p

    

In [ ]:
x = np.arange(-10, 10, 0.1)
y = np.arange(-10, 10, 0.1)

xx, yy = np.meshgrid(x, y, sparse=False)

p_array = np.zeros(xx.shape)

for i in range(p_array.shape[0]):
    for j in range(p_array.shape[1]):
        
        x = [xx[i,j], yy[i,j]]
        
        p_array[i,j] = predictive_posterior_prob(x, mu_N, sigma_N)
        
        

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(p_array, origin='lower')
plt.show()
plt.figure(figsize=(10,10))
plt.scatter(gaussian_1[:,0], gaussian_1[:,1])
plt.scatter(gaussian_2[:,0], gaussian_2[:,1])
plt.xlim(-10,10)
plt.ylim(-10,10)
plt.gca().set_aspect('equal', adjustable='box')
plt.show()